In [0]:
from google.colab import drive

In [20]:
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:

%cd drive 


/content/drive


In [29]:
!ls


'My Drive'


In [30]:
%cd 'My Drive'


/content/drive/My Drive


In [31]:
%cd 'CGM Project'

/content/drive/My Drive/CGM Project


In [0]:
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import math

In [0]:
df = pd.read_csv('Charades_v1_train.csv')
path = '/charades_v1_rgb/'
dir_path = os.getcwd() + path
ar = next(os.walk(dir_path))[1]
#ar = ['0ZOPH', 'T7449', 'BA6DD', 'OVHFT', 'SH20Q', '5L04F', 'O2VP8', 'GPR89', '9EEGQ', 'PWR6D', '0RP6B', '0OP1K']


In [0]:
def load_image( infilename ) :
    img = Image.open( infilename )
    img.load()
    data = np.asarray( img, dtype="int32" )
    return data

In [0]:
feature_extractor_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2" #@param {type:"string"}


In [0]:
from scipy import ndimage, misc
import tensorflow_hub as hub
import numpy as np

feature_extractor_layer = hub.KerasLayer(feature_extractor_url, input_shape=(224,224,3))


In [0]:
action_frames = {}

for j in range(len(ar)):
  action_frames[ar[j]] = {}

for i in range(len(df)):
  for j in range(len(ar)):
    if(df.loc[i, "id"] == ar[j]):
      actions = df.loc[i, "actions"].split(";")
      v_id = df.loc[i, "id"]
      frames = []
      video_path = os.getcwd() + path + v_id
      print(video_path)
      for r, d, f in os.walk(video_path):     
        for files in f:
          nar = load_image(video_path + '/' + files)
          frames.append(nar)
      frames = frames[:-1]
      req_frames = {}
      for k in range(len(actions)):
        label, start, end = actions[k].split(' ')
        req_frames[label] = []
      for p in range(len(frames)):
        for k in range(len(actions)):
          label, start, end = actions[k].split(' ')
          startframe = math.floor(float(start) * 24)
          endframe = math.floor(float(end) * 24)
          if(p >= startframe and p <= endframe):
            req_frames[label].append(frames[p])
      imp_frames = {}
      for k in range(len(actions)):
        label, start, end = actions[k].split(' ')
        imp_frames[label] = []
        for p in range(len(req_frames[label])):
          matches = 0
          for q in range(len(imp_frames[label])):
            if((np.sum(np.bitwise_xor(req_frames[label][p], imp_frames[label][q])) / np.sum(req_frames[label][p])) < 0.2):
              matches += 1
              break
          if(matches == 0):
            imp_frames[label].append(req_frames[label][p])
        print(len(req_frames[label]), len(imp_frames[label]))
      action_frames[ar[j]] = imp_frames
print(action_frames)


              
  

/content/drive/My Drive/CGM Project/charades_v1_rgb/OVHFT
241 46
225 45
159 42
189 37
174 38
189 37
189 37
128 31
171 40
128 35
/content/drive/My Drive/CGM Project/charades_v1_rgb/PWR6D
644 83
744 86
238 57
560 79
744 86
/content/drive/My Drive/CGM Project/charades_v1_rgb/0ZOPH
150 50
527 108
101 43
121 44
201 67
/content/drive/My Drive/CGM Project/charades_v1_rgb/5L04F
692 1
390 1
178 1
390 1
692 1
692 1
/content/drive/My Drive/CGM Project/charades_v1_rgb/BA6DD
826 140
/content/drive/My Drive/CGM Project/charades_v1_rgb/GPR89
140 24
281 30
620 48
347 35
473 39
253 26
1294 63
140 24
/content/drive/My Drive/CGM Project/charades_v1_rgb/T7449
651 18
651 18
651 18
644 19
651 18
/content/drive/My Drive/CGM Project/charades_v1_rgb/0RP6B
556 44
201 34
539 43
248 34
167 33
123 29
145 26
114 25
237 39
635 44
635 44
/content/drive/My Drive/CGM Project/charades_v1_rgb/O2VP8
217 160
174 137
646 275
117 103
481 251


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [33]:
import tensorflow as tf

imb = tf.placeholder(dtype=tf.float32, shape=(None,224,224,3))
feature_batch = feature_extractor_layer(imb)
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
#temp = sess.run(feature_batch, feed_dict= {imb: image_batch})

NameError: ignored

In [0]:
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

ma =250
file_batch = np.zeros(shape=(1,ma,1280))
for a in ar:
  image_batch = np.zeros(shape=(1,224,224,3))
  for k, action in action_frames[a].items():
    #print(len(action))
    for images in action:
      #print(np.shape(images))
      #plt.imshow(images)
      #plt.show()
      im = np.resize(images,[224,224,3])
      #print(np.shape(im))
      im = np.expand_dims(im, axis = 0)
      #print(np.shape(im))
      image_batch = np.vstack((image_batch,im))
      #print(np.shape(image_batch))
    break
  image_batch = image_batch[1:]
  temp = sess.run(feature_batch, feed_dict= {imb: image_batch})
  stk=np.zeros(shape = (ma-temp.shape[0],1280))
  new_temp = np.vstack((stk, temp))
  new_temp = np.expand_dims(new_temp, 0)
  file_batch = np.vstack((file_batch,new_temp))
  #print(file_batch.shape)
    

  


(2, 224, 224, 3)
(3, 224, 224, 3)
(4, 224, 224, 3)
(5, 224, 224, 3)
(6, 224, 224, 3)
(7, 224, 224, 3)
(8, 224, 224, 3)
(9, 224, 224, 3)
(10, 224, 224, 3)
(11, 224, 224, 3)
(12, 224, 224, 3)
(13, 224, 224, 3)
(14, 224, 224, 3)
(15, 224, 224, 3)
(16, 224, 224, 3)
(17, 224, 224, 3)
(18, 224, 224, 3)
(19, 224, 224, 3)
(20, 224, 224, 3)
(21, 224, 224, 3)
(22, 224, 224, 3)
(23, 224, 224, 3)
(24, 224, 224, 3)
(25, 224, 224, 3)
(26, 224, 224, 3)
(27, 224, 224, 3)
(28, 224, 224, 3)
(29, 224, 224, 3)
(30, 224, 224, 3)
(31, 224, 224, 3)
(32, 224, 224, 3)
(33, 224, 224, 3)
(34, 224, 224, 3)
(35, 224, 224, 3)
(36, 224, 224, 3)
(37, 224, 224, 3)
(38, 224, 224, 3)
(39, 224, 224, 3)
(40, 224, 224, 3)
(41, 224, 224, 3)
(42, 224, 224, 3)
(43, 224, 224, 3)
(44, 224, 224, 3)
(45, 224, 224, 3)
(46, 224, 224, 3)
(47, 224, 224, 3)
(48, 224, 224, 3)
(49, 224, 224, 3)
(50, 224, 224, 3)
(51, 224, 224, 3)
(2, 250, 1280)
(2, 224, 224, 3)
(3, 224, 224, 3)
(4, 224, 224, 3)
(5, 224, 224, 3)
(6, 224, 224, 3)
(7, 224,

In [0]:
file_batch.shape

(13, 250, 1280)

In [0]:
np.save("./image_embedding.npy", file_batch)

In [42]:
np.load("./image_embedding_bak.npy")[3]

IndexError: ignored

In [0]:
import tensorflow as tf


In [0]:
tf.__version__

'1.15.0'